In [7]:
%matplotlib inline  

from os.path import isfile, join, exists
from os import listdir, makedirs, walk, remove, getlogin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from nilearn import image, masking
import scipy.stats as stats
import os
import numpy as np
import pickle
import librosa

In [8]:
# exp params
projdir = '/mnt/bucket/labs/hasson/mai/projects/b2b_teaching'
    
run_list = ['teach1_epi_lesson1', 'teach2_epi_lesson2', 'teach2_epi_lesson3', 
            'teach3_epi_lesson4', 'teach3_epi_lesson5']

hrf = 3
model_epi = 'teach2_epi_lesson3'

In [9]:
# get time stamps
x1 = pd.ExcelFile(join(projdir, 'recordings', 'lesson_timestamps.xlsx'))
df_times = x1.parse('times')

df_times = df_times.drop([0])
timestamps = {};
for i,run in enumerate(run_list):
    trs = df_times['tr' + str(i+1)]
    timestamps[run] = trs.tolist()

for run in run_list:
    times = timestamps[run]
    for i in range(len(times)-1):
        this_time = times[i]
        next_time = times[i+1]
        if next_time <= this_time:
            print('out of order: run ' + run + ', ind ' + str(i) + ', tr ' + str(times[i]))


In [10]:
# get times for model epi
event_len = []
model_times = timestamps[model_epi]

for i in range(len(model_times)-1):
    time_len = model_times[i+1] - model_times[i]
    event_len.append(int(time_len))

# get last event time
datafile = join(projdir, 'teach_data', 'data', model_epi + '_data.pk1')
pkl_file = open(datafile, 'rb')
datadict = pickle.load(pkl_file)
pkl_file.close()
subdata = datadict['data']

# last time
last_len = subdata.shape[1] - model_times[-1]
event_len.append(int(last_len))


In [105]:
#run = run_list[0]

for run in run_list:
    print(run + '...')
    # load data
    datafile = join(projdir, 'teach_data', 'data', run + '_data.pk1')
    pkl_file = open(datafile, 'rb')
    datadict = pickle.load(pkl_file)
    pkl_file.close()
    subdata = datadict['data']

    # mask data so only non-nan voxels
    goodvox_mask = ~np.isnan(np.mean(subdata,axis=1))
    subdata_good = subdata[goodvox_mask,:]

    # event times for this run
    times = timestamps[run]

    # get event data and stretch
    for i in range(len(times)):

        # get event data
        start = int(times[i]) + hrf
        if i == len(times)-1:
            stop = subdata_good.shape[1]
        else:
            stop = int(times[i+1]) + hrf
        ev_orig = subdata_good[:, start:stop]

        # stretch
        ev_stretch = librosa.core.resample(ev_orig, ev_orig.shape[1], event_len[i])

        # build up timeseries
        if i == 0:
            sub_stretch = ev_stretch
        else:
            sub_stretch = np.concatenate([sub_stretch, ev_stretch], axis=1)

    # unmask-put back in original order
    subdata_stretch = np.full([subdata.shape[0], sub_stretch.shape[1]], np.nan)
    subdata_stretch[goodvox_mask,:] = sub_stretch
    
    # save
    datadict['data'] = subdata_stretch
    datadict['stretch'] = 1
    datadict['modelEPI'] = model_epi
    datadict['origDatafile'] = datafile
    datadict['hrf'] = hrf
    
    savename = join(projdir, 'teach_data', 'data', run + '_data_hrf' + str(hrf) + '_stretch.pk1')
    output = open(savename, 'wb')
    pickle.dump(datadict, output)
    output.close()

print('done')

teach1_epi_lesson1...
teach2_epi_lesson2...
teach2_epi_lesson3...
teach3_epi_lesson4...
teach3_epi_lesson5...
done


In [11]:
# load data
data_list = []
for run in run_list:
    datafile = join(projdir, 'teach_data', 'data', run + '_data_hrf' + str(hrf) + '_stretch.pk1')
    pkl_file = open(datafile, 'rb')
    datadict = pickle.load(pkl_file)
    pkl_file.close()

    # get data
    data_list.append(datadict['data'])

# convert data to numpy array
data = np.asarray(data_list)

# get sum of all
data_sum = np.nansum(data, axis=0)

In [22]:

# do isc
r_all = []
for i,subdata in enumerate(data):
    print(run_list[i] + '...')
    avg_others = (data_sum - subdata) / (data.shape[0]-1)
    r = corr_fast(subdata, avg_others)
    r_all.append(r)


teach1_epi_lesson1...


/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2245: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


teach2_epi_lesson2...
teach2_epi_lesson3...
teach3_epi_lesson4...
teach3_epi_lesson5...


In [30]:
r_all = np.asarray(r_all)
isc = np.nanmean(r_all, axis=0)

# convert to nifti
mask_nifti = image.load_img(datadict['mask'])
data_unmask = masking.unmask(isc, mask_nifti)

# save nifti
savename = join(projdir, 'teach_data', 'analysis', 'isc_teachData.nii.gz')
data_unmask.to_filename(savename)


In [31]:
savename

'/mnt/bucket/labs/hasson/mai/projects/b2b_teaching/teach_data/analysis/isc_teachData.nii.gz'

In [15]:
def corr_fast(mat1, mat2):
# Vectorized method for correlating rows of one matrix with another
    
    # zscore
    mat1_z = stats.zscore(mat1, axis=1, ddof=1)
    mat2_z = stats.zscore(mat2, axis=1)
            
    # Calculate sum of products
    sum_vec = np.nansum(np.transpose(mat1_z) * np.transpose(mat2_z),
                        axis = 0)
                        
    # Calculate degrees of free
    dof = mat1.shape[1] - 1
    
    # Correlation coeff
    r = sum_vec/dof        
    return r

In [100]:
print(subdata.shape)
print(subdata_good.shape)
print(sub_stretch.shape)
print(subdata_stretch.shape)

(65323, 1035)
(61417, 1035)
(61417, 978)
(65323, 978)


In [103]:
datadict['mask']

'/mnt/bucket/labs/hasson/mai/standard/MNI_brains/MNI152_T1_3mm_brain_mask_bin.nii'